<a href="https://colab.research.google.com/github/kirath2205/Final-Year-Project/blob/Test_bench/Test_bench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras,os
import tensorflow as tf
from keras.models import Sequential,Model
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D , Dropout
import numpy as np
from tensorflow import keras
from keras.initializers import glorot_uniform
from keras import applications
from tensorflow.keras.applications.resnet50 import ResNet50 , preprocess_input


In [2]:
!pip install Keras-applications


In [3]:
def select_dataset(index):

  if(index==1):
    fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
    (X_train, y_train), (X_test, y_test) = fashion_mnist

  elif(index==2):
    cifar_10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar_10
  
  elif(index==3):
    cifar_100 = tf.keras.datasets.cifar100.load_data()
    (X_train, y_train), (X_test, y_test) = cifar_100
    print(len(X_train))

    
  return X_train,y_train,X_test,y_test

In [4]:
def pre_processing_fashion_mnist(X_train,y_train,X_test,y_test,image_channel):
  img_width = X_train.shape[1]
  img_height = X_train.shape[2]
  input_shape = (img_width, img_height, image_channel)
  
  # normalize data
  X_train, X_test = X_train / 255, X_test / 255

  # reshape input 
  X_train = X_train.reshape(X_train.shape[0], *input_shape)
  X_test = X_test.reshape(X_test.shape[0], *input_shape)

  # one-hot
  y_train = tf.keras.utils.to_categorical(y_train)
  y_test = tf.keras.utils.to_categorical(y_test)

  return X_train,y_train,X_test,y_test,input_shape

def pre_processing_cifar10(X_train,y_train,X_test,y_test,image_channel):
  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')

  y_train = keras.utils.to_categorical(y_train)
  y_test = keras.utils.to_categorical(y_test)

  img_width = X_train[0].shape[0]
  img_height = X_train[0].shape[1]
  image_channel = image_channel

  input_shape=(img_width,img_height,image_channel) 
  print(input_shape)
  return X_train,y_train,X_test,y_test,input_shape
  
def pre_processing_cifar100(X_train,y_train,X_test,y_test,image_channel):
  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')
  X_train = preprocess_input(X_train)
  X_test = preprocess_input(X_test)

  y_train = keras.utils.to_categorical(y_train)
  y_test = keras.utils.to_categorical(y_test)

  img_width = X_train[0].shape[0]
  img_height = X_train[0].shape[1]
  image_channel = image_channel

  input_shape=(img_width,img_height,image_channel) 
  print(input_shape)
  return X_train,y_train,X_test,y_test,input_shape

In [5]:
def define_model_vgg16(image_shape,total_classes):

  model = Sequential()
  model.add(Conv2D(input_shape=image_shape,filters=64,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'))
  model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
  model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
  model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
  model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'))
  model.add(Flatten())
  model.add(Dense(units=4096,activation="relu"))
  model.add(Dense(units=4096,activation="relu"))
  model.add(Dense(units=total_classes, activation="softmax"))
  opt = keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
  
  return model

def define_model_standard_cnn(image_shape,total_classes):
  initializer = tf.keras.initializers.HeUniform()
  model = Sequential()
  model.add(Conv2D(32,(3,3), activation = 'relu',  padding = 'same', input_shape = image_shape))
  model.add(Conv2D(32,(3,3), activation = 'relu',  padding = 'same'))
  model.add(MaxPooling2D((2,2)))
  model.add(Dropout(0.2))  
  model.add(Conv2D(64,(3,3), activation = 'relu',  padding = 'same'))
  model.add(Conv2D(64,(3,3), activation = 'relu',  padding = 'same'))
  model.add(MaxPooling2D((2,2)))
  model.add(Dropout(0.2))
  model.add(Conv2D(128,(3,3), activation = 'relu',  padding = 'same'))
  model.add(Conv2D(128,(3,3), activation = 'relu',  padding = 'same'))
  model.add(MaxPooling2D((2,2)))
  model.add(Dropout(0.2))
  model.add(Flatten())
  model.add(Dense(128, activation = 'relu'))
  model.add(Dense(total_classes, activation = 'softmax'))
  opt = keras.optimizers.SGD(learning_rate=0.001 , momentum=0.9)
  model.compile(optimizer=opt,loss=keras.losses.categorical_crossentropy,metrics=['accuracy'])
  return model



In [6]:
def identity_block(layer,f,nfilters):
    Xshortcut=layer
    F1,F2,F3=nfilters
    layer=Conv2D(F1,(1,1),padding='valid')(layer)
    layer=BatchNormalization(axis=-1)(layer)
    layer=Activation('relu')(layer)
    
    layer=Conv2D(F2,(f,f),padding='same')(layer)
    layer=BatchNormalization(axis=-1)(layer)
    layer=Activation('relu')(layer)
    
    layer=Conv2D(F3,(1,1),padding='valid')(layer)
    layer=BatchNormalization(axis=-1)(layer)
    
    layer=Add()([Xshortcut,layer])
    layer=Activation('relu')(layer)
    return(layer)

def Conv_block(layer,f,nfilters,s=2):
    Xshortcut=layer
    F1,F2,F3=nfilters
    layer=Conv2D(F1,(1,1),strides=(s,s))(layer)
    layer=BatchNormalization(axis=-1)(layer)
    layer=Activation('relu')(layer)
    
    layer=Conv2D(F2,(f,f),padding='same')(layer)
    layer=BatchNormalization(axis=-1)(layer)
    layer=Activation('relu')(layer)
    
    layer=Conv2D(F3,(1,1))(layer)
    layer=BatchNormalization(axis=-1)(layer)
    
    Xshortcut=Conv2D(F3,(1,1),strides=(s,s))(Xshortcut)
    Xshortcut=BatchNormalization(axis=-1)(Xshortcut)
    
    layer=Add()([Xshortcut,layer])
    layer=Activation('relu')(layer)
    return(layer)

def Resnet50(img_shape,numclasses):
    img_inp=Input(shape=img_shape,dtype=tf.float32)
    X=ZeroPadding2D((3,3))(img_inp)
    X=Conv2D(64,(7,7),strides=(2,2))(X)
    X=BatchNormalization(axis=-1)(X)
    X=Activation('relu')(X)
    
    X=MaxPooling2D((3,3),strides=(2,2),padding='same')(X)
    X=Conv_block(X,3,[64,64,256],s=1)
    X=identity_block(X,3,[64,64,256])
    X=identity_block(X,3,[64,64,256])
    
    X=Conv_block(X,3,[128,128,512],s=2)
    X=identity_block(X,3,[128,128,512])
    X=identity_block(X,3,[128,128,512])
    X=identity_block(X,3,[128,128,512])
    
    X=Conv_block(X,3,[256,256,1024],s=2)
    X=identity_block(X,3,[256,256,1024])
    X=identity_block(X,3,[256,256,1024])
    X=identity_block(X,3,[256,256,1024])
    X=identity_block(X,3,[256,256,1024])
    X=identity_block(X,3,[256,256,1024])
    
    X=Conv_block(X,3,[512,512,2048],s=2)
    X=identity_block(X,3,[512,512,2048])
    X=identity_block(X,3,[512,512,2048])
    
    X=AveragePooling2D((7,7) , padding='same')(X)
    
    X=Flatten()(X)
    X=Dense(numclasses,activation='softmax')(X)
    model=Model(inputs=img_inp,outputs=X)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return(model)

In [7]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
def train_model_vgg16(X_train,y_train,X_test,y_test,epochs=10,batch_size=256):
  
  callbacks = [ 
    tf.keras.callbacks.ModelCheckpoint('best_model_vgg16', monitor='val_accuracy', verbose=1, save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau( factor = 0.1, patience = 3, min_lr = 0.00001, verbose = 1 )
  ]
  history=model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size,callbacks = callbacks, verbose = 1,validation_data=(X_test,y_test))
  return history

def train_model_standard_cnn(X_train,y_train,X_test,y_test,epochs=10,batch_size=64):
  callbacks = [ 
    tf.keras.callbacks.ModelCheckpoint('best_model_standard_cnn', monitor='val_accuracy', verbose=1, save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau( factor = 0.1, patience = 3, min_lr = 0.00001, verbose = 1 )
  ]
  history=model.fit(X_train , y_train , epochs = epochs , validation_data=(X_test,y_test),batch_size=batch_size , callbacks = callbacks)
  return history

def train_model_resnet50(X_train,y_train,X_test,y_test,epochs=10):
  callbacks = [ 
    tf.keras.callbacks.ModelCheckpoint('best_model_resnet50', monitor='val_accuracy', verbose=1, save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau( factor = 0.1, patience = 3, min_lr = 0.00001, verbose = 1 )
  ]
  history=model.fit(X_train , y_train , epochs = epochs , validation_data=(X_test,y_test),batch_size=32 , callbacks = callbacks)
  return history

In [8]:
def make_prediction(model,X_test,index):

  return model.predict(X_test(index))


In [9]:

'''X_train,y_train,X_test,y_test=select_dataset(1)
X_train,y_train,X_test,y_test,image_shape=pre_processing_fashion_mnist(X_train,y_train,X_test,y_test,1)
model=define_model(image_shape,10)
train_model(X_train,y_train,X_test,y_test)'''

'''X_train,y_train,X_test,y_test=select_dataset(2)
X_train,y_train,X_test,y_test,image_shape=pre_processing_cifar10(X_train,y_train,X_test,y_test,3)
model=define_model_standard_cnn(image_shape,10)
train_model_standard_cnn(X_train,y_train,X_test,y_test,epochs=75)'''

'''X_train,y_train,X_test,y_test=select_dataset(3)
X_train,y_train,X_test,y_test,image_shape=pre_processing_cifar100(X_train,y_train,X_test,y_test,3)
model=Resnet50(image_shape,100)
history=train_model_resnet50(X_train,y_train,X_test,y_test,epochs=100)'''



(32, 32, 3)
Epoch 1/75
782/782 [==============================] - 19s 22ms/step - loss: 2.1687 - accuracy: 0.2175 - val_loss: 1.8917 - val_accuracy: 0.3248

Epoch 00001: val_accuracy improved from -inf to 0.32480, saving model to best_model_standard_cnn
INFO:tensorflow:Assets written to: best_model_standard_cnn/assets
Epoch 2/75
782/782 [==============================] - 16s 21ms/step - loss: 1.7922 - accuracy: 0.3434 - val_loss: 1.6457 - val_accuracy: 0.4157

Epoch 00002: val_accuracy improved from 0.32480 to 0.41570, saving model to best_model_standard_cnn
INFO:tensorflow:Assets written to: best_model_standard_cnn/assets
Epoch 3/75
782/782 [==============================] - 17s 21ms/step - loss: 1.5867 - accuracy: 0.4179 - val_loss: 1.4876 - val_accuracy: 0.4698

Epoch 00003: val_accuracy improved from 0.41570 to 0.46980, saving model to best_model_standard_cnn
INFO:tensorflow:Assets written to: best_model_standard_cnn/assets
Epoch 4/75
782/782 [==============================] - 16s 

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

# Visualize history
# Plot history: Loss
plt.plot(history.history['val_loss'])
plt.title('Validation loss history')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.show()

# Plot history: Accuracy
plt.plot(history.history['val_accuracy'])
plt.title('Validation accuracy history')
plt.ylabel('Accuracy value (%)')
plt.xlabel('No. epoch')
plt.show()

In [ ]:
!pip freeze > requirements.txt